In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
def load_data(data_path):

  with open(data_path) as training_file:
    csv_reader=csv.reader(training_file,delimiter=',')
    first_line=True
    temp_images=[]
    temp_labels=[]

    for row in csv_reader:
      if first_line:
        first_line=False
      else:
        temp_labels.append(row[0])
        image_data=row[1:]
        image_data_as_array=np.array_split(image_data,28)
        temp_images.append(image_data_as_array)
    images=np.array(temp_images).astype('float')
    labels=np.array(temp_labels).astype('float')

    return images,labels

In [4]:
def load_az_dataset(datasetPath):
  data=[]
  labels=[]

  for row in open(datasetPath):
    row=row.split(",")
    label=int(row[0])
    image=np.array([int(x) for x in row[1:]])
    image=image.reshape((28,28))

    data=np.array(data.append(image),dtype="float32")
    labels=np.array(labels.append(labels))

    return data, labels

In [5]:
def load_mnist_dataset():
  ((trainData,trainLabels),(testData,testLabels))=mnist.load_data()
  data=np.vstack([trainData,testData])
  labels=np.hstack([trainLabels,testLabels])

  return (data,labels)

In [6]:
import matplotlib
matplotlib.use("Agg")

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import *

In [9]:
Epochs=50
init_lr=1e-1
batch_size=128

print("Loading datasets.....")

(azData, azLabels)=load_data('/content/drive/My Drive/Colab Notebooks/Py_image_search_OCR/A_Z Handwritten Data.csv')
(digitsData,digitsLabels)=load_mnist_dataset()

Loading datasets.....
11493376/11490434 [==============================] - 0s 0us/step


In [10]:
azData.shape

(372450, 28, 28)

In [11]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Py_image_search_OCR/A_Z Handwritten Data.csv')

In [12]:
df.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,...,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617,0.618,0.619,0.620,0.621,0.622,0.623,0.624,0.625,0.626,0.627,0.628,0.629,0.630,0.631,0.632,0.633,0.634,0.635,0.636,0.637,0.638,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
azLabels+=10

data=np.vstack([azData,digitsData])
labels=np.hstack([azLabels,digitsLabels])

In [14]:
data=[cv2.resize(image,(32,32)) for image in data]
data=np.array(data,dtype='float32')

data=np.expand_dims(data,axis=-1)
data/=255.0

In [15]:
le=LabelBinarizer()
labels=le.fit_transform(labels)
counts=labels.sum(axis=0)

In [16]:
classTotals=labels.sum(axis=0)
classWeight={}

for i in range(0, len(classTotals)):
  classWeight[i]=classTotals.max()/classTotals[i]

In [17]:
(trainX, testX, trainY, testY)=train_test_split(data,
              labels,test_size=0.2,stratify=labels,random_state=42)

In [18]:
Datagen=ImageDataGenerator(
    rotation_range=10,
	  zoom_range=0.05,
	  width_shift_range=0.1,
	  height_shift_range=0.1,
	  shear_range=0.15,
	  horizontal_flip=False,
	  fill_mode="nearest"
)

In [19]:
opt=SGD(lr=1e-1, decay=(1e-1)/50)

model=ResNet50(weights='imagenet',include_top=False,input_shape=(32,32,3))

x=model.output
x=Flatten()(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(36,activation='softmax')(x)
modelFinal=Model(inputs=model.input,outputs=x)
for layer in model.layers:
 layer.trainable = False

modelFinal.compile(loss="categorical_crossentropy",optimizer=opt,
              metrics=['accuracy'])

94773248/94765736 [==============================] - 1s 0us/step


In [20]:
model_checkpoint = ModelCheckpoint('Trained_Model.hdf5',
                                    monitor='val_accuracy',
                                    verbose=1,
                                    mode='max',
                                    save_best_only=True)

In [21]:
def large_model():
  model=Sequential()
  model.add(Conv2D(30,(5,5),input_shape=(32,32,1),activation='relu'))
  model.add(MaxPooling2D())
  model.add(Conv2D(15,(3,3),activation='relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128,activation='relu'))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(36,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [22]:
large_model.compile(loss="categorical_crossentropy",optimizer=opt,
              metrics=['accuracy'])

AttributeError: ignored

In [23]:
model=large_model()
H=model.fit(
    trainX,
    trainY,
    batch_size=128,
    validation_data=(testX,testY),
    steps_per_epoch=len(trainX)//128,
    epochs=50,
    class_weight=classWeight,
    verbose=1,
    callbacks=[model_checkpoint])

Epoch 1/50
2763/2765 [============================>.] - ETA: 0s - loss: 1.8145 - accuracy: 0.8639
Epoch 00001: val_accuracy improved from -inf to 0.94290, saving model to Trained_Model.hdf5
2765/2765 [==============================] - 13s 5ms/step - loss: 1.8139 - accuracy: 0.8639 - val_loss: 0.1790 - val_accuracy: 0.9429
Epoch 2/50
2763/2765 [============================>.] - ETA: 0s - loss: 0.7307 - accuracy: 0.9340
Epoch 00002: val_accuracy improved from 0.94290 to 0.95621, saving model to Trained_Model.hdf5
2765/2765 [==============================] - 12s 5ms/step - loss: 0.7304 - accuracy: 0.9341 - val_loss: 0.1421 - val_accuracy: 0.9562
Epoch 3/50
2759/2765 [============================>.] - ETA: 0s - loss: 0.5914 - accuracy: 0.9449
Epoch 00003: val_accuracy did not improve from 0.95621
2765/2765 [==============================] - 12s 5ms/step - loss: 0.5913 - accuracy: 0.9449 - val_loss: 0.1272 - val_accuracy: 0.9539
Epoch 4/50
2760/2765 [============================>.] - ETA: 0

In [24]:
labelNames="0123456789"
labelNames+="ABCDEFGHIJKLMNOPQURSUVWXYZ"
labelNames=[l for l in labelNames]

In [25]:
len(labelNames)

36

In [26]:
predictions=model.predict(testX,batch_size=128)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1),target_names=labelNames))

              precision    recall  f1-score   support

           0       0.89      0.83      0.86      1381
           1       0.99      0.99      0.99      1575
           2       0.97      0.98      0.98      1398
           3       0.99      0.99      0.99      1428
           4       0.99      0.98      0.98      1365
           5       0.95      0.96      0.95      1263
           6       0.99      0.99      0.99      1375
           7       0.98      0.99      0.99      1459
           8       0.98      0.99      0.98      1365
           9       0.99      0.98      0.99      1392
           A       0.99      1.00      0.99      2774
           B       0.99      0.99      0.99      1734
           C       0.99      0.99      0.99      4682
           D       0.95      0.99      0.97      2027
           E       0.99      1.00      0.99      2288
           F       0.99      0.99      0.99       232
           G       0.97      0.99      0.98      1152
           H       0.98    

In [27]:
N=np.arange(0,50)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig(args["plot"])
plt.show()

In [28]:
# Plot training & validation accuracy values
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid()
plt.show()

# Plot training & validation loss values
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid()
plt.show()